In [1]:
import pandas as pd
import numpy as np
import os
from itertools import product
import string
import regex
import re
import random
from collections import OrderedDict, defaultdict
from operator import itemgetter
from collections import Counter
from toolz.dicttoolz import valmap
from tqdm import tqdm
from itertools import product, zip_longest
import functools
import copy

## Part 1

In [2]:
russian_pattern = r'[^\p{Cyrillic} ]'
alphabet = [*'абвгдеёжзийклмнопрстуфхцчшщъыьэюя ']

In [3]:
def preprocess_string(line, language_pattern=russian_pattern):
    line = line.strip().lower().translate(str.maketrans('', '', string.punctuation))
    line = line.replace('\t', '').replace('\n', '').replace('ї', '').replace('–', '').replace('—', '')
    line = regex.sub(language_pattern, '', line)
    line = line.translate(str.maketrans('', '', string.digits))
    line = re.sub(' +', ' ', line)
    
    return line

In [4]:
def read_file_text(text_path):
    text = []

    with open(text_path) as f:
        for line in f.readlines():
            line = line.strip().lower().translate(str.maketrans('', '', string.punctuation))
            line = line.replace('\t', '').replace('\n', '').replace('–', '').replace('—', '')

            if len(line):
                text.append(line)
    return text

In [5]:
def read_preprocess_file_text(text_path, language_pattern=russian_pattern):
    text = read_file_text(text_path)
    text = [preprocess_string(line, language_pattern) for line in text]        
    
    return preprocess_string(' '.join(text))

In [6]:
def dict_to_ordered_sorted(dictionary, key=itemgetter(1), reverse=True):
    return OrderedDict(sorted(dictionary.items(), key=key, reverse=reverse))

In [7]:
def calculate_frequency_for_string(text, n_gram=1):
    n_gram_alphabet = set(''.join(n_gram) for n_gram in product(alphabet, repeat=n_gram))
    
    frequency = {}

    for n_gram in n_gram_alphabet:
        frequency[n_gram] = text.count(n_gram)
        
    return dict_to_ordered_sorted(frequency)

In [8]:
def encrypt(text, frequency_dict, n_gram=1):
    encrypt = {orig: enc for orig, enc in zip(frequency_dict.keys(), random.sample(list(frequency_dict.keys()), k=len(frequency_dict.keys())))}
    return ''.join([encrypt[text[i:i + n_gram]] for i in range(0, len(text), n_gram)])

In [9]:
def decrypt(encrypted_text, frequency_dict, n_gram=1):
    frequency_encrypted = calculate_frequency_for_string(encrypted_text, n_gram)
    decrypt = {enc: orig for enc, orig in zip(frequency_encrypted.keys(), frequency_dict.keys())}
    
    return ''.join([decrypt[encrypted_text[i:i + n_gram]] for i in range(0, len(encrypted_text), n_gram)])

In [10]:
def cut_text(text, n_gram=1):
    text_len = len(text)

    while(text_len % n_gram != 0):
        text_len -= 1
        
    return text[:text_len]

In [11]:
def decryption_accuracy(original_text, decrypted_text):
    correctly_decrypted = 0
    
    for orig, dec in zip_longest(original_text, decrypted_text, fillvalue=''):
        if orig == dec:
            correctly_decrypted += 1
    
    return correctly_decrypted / len(original_text)

In [12]:
def encrypt_decrypt(text, frequency_dict, n_gram=1, show_n=100):
    text = cut_text(text, n_gram)
    
    print('Original text: \n')
    print(text[:show_n], '\n')
    
    encrypted_text = encrypt(text, frequency_dict, n_gram)
    
    print('Encrypted text: \n' )
    print(encrypted_text[:show_n], '\n')
    
    decrypted_text = decrypt(encrypted_text, frequency_dict, n_gram)
    
    print('Decrypted text: \n')
    print(decrypted_text[:show_n], '\n')
    
    print(f'Correctly decrypted percentage: {decryption_accuracy(text, decrypted_text)}')

In [13]:
text_rus = read_preprocess_file_text('./corpora/WarAndPeace.txt')

In [14]:
frequency_rus = calculate_frequency_for_string(text_rus)

In [15]:
test_text = read_preprocess_file_text('./corpora/AnnaKarenina.txt')

In [16]:
test_string = preprocess_string('свойство или приспособление организма, потенциально имеющее адаптивную (приспособительную) ценность. Теория преадаптации позволяет описать механизм смены функций органов в процессе эволюции и разрешить парадокс образования органов, конечная функция которых не имела первоначально приспособительной ценности. Концепция преадаптации позволяет разрешить одну из проблем эволюционной теории: невозможность развития сложных приспособлений, которые способны эффективно функционировать, лишь будучи хорошо сформированными. Подобные органы, находясь в зачаточном состоянии, не могут повышать приспособленность организма, а значит, казалось бы, и не могут появиться эволюционным путём. Примерами таких органов являются челюстной аппарат, среднее ухо, плавательный пузырь и др. Идея преадаптации состоит в том, что многие органы и приспособления сформировались, первоначально выполняя иные функции, чем на конечной стадии своего развития. В какой-то момент орган начинает выполнять дополнительную функцию, которая оказывается более ценной, чем первоначальная. В результате эволюция органа (путём естественного отбора) подчиняется требованию улучшения новой функции. Изменение органа в новом направлении может привести к утрате прежней функции. Таким образом, сложные органы даже на начальных стадиях своего эволюционного развития имеют приспособительную ценность, но их начальная функция может быть другой (в тех случаях, когда с точки зрения конечной функции приспособительной ценности нет, тогда и возникает вышеописанный парадокс)')

In [17]:
encrypt_decrypt(test_string, frequency_rus)

Original text: 

свойство или приспособление организма потенциально имеющее адаптивную приспособительную ценность тео 

Encrypted text: 

атгюачтгиыэыисяыасгагбэнмынигявемыхшеисгчнм ыеэумгиышнёпнниежесчытмдёисяыасгагбычнэумдёи нммгачуичнг 

Decrypted text: 

лрожлтро ака всалволойкнеан осзиеаыми вотнегаикуео амншённ ибивтареяш всалволойатнкуеяш гнееолту тно 

Correctly decrypted percentage: 0.3142664872139973


In [18]:
encrypt_decrypt(test_text, frequency_rus)

Original text: 

анна каренина один из самых знаменитых романов льва толстого начинается ставшей афоризмом фразой все 

Encrypted text: 

чббчфсчхйбшбчфзншбфшыфючолтфыбчойбшьлтфхзочбзкфэукчфьзэюьзщзфбчцшбчйьюефюьчкгйвфчпзхшыозофпхчызвфкюй 

Decrypted text: 

аииа кареиниа одни нч самьх чиамеинтьх ромаиов лгва толстоыо иазниается ставшеж афорнчмом фрачож все 

Correctly decrypted percentage: 0.7855274449249966


#### Даже на большом тестовом тексте получилась какая-то грусть, хотя, если постараться, большинство слов распознать все равно можно

## Part 2

In [19]:
bigram_frequency_rus = calculate_frequency_for_string(text_rus, n_gram=2)

In [20]:
encrypt_decrypt(test_string, bigram_frequency_rus, n_gram=2)

Original text: 

свойство или приспособление организма потенциально имеющее адаптивную приспособительную ценность тео 

Encrypted text: 

яёэочэикяезеснекрльыт шельпзерззльеэбркчьёйигпъжвэяе зэесвхищщмечрйкшсти цкчйээжьёъжйкшсёпсхьытёфяпу 

Decrypted text: 

рунина ву вео нороь же ли та неси зол  о тбуящска у в внвуюшриоемупаслрачт оерым тскпаслхадаь запомл 

Correctly decrypted percentage: 0.06123822341857335


In [21]:
encrypt_decrypt(test_text, bigram_frequency_rus, n_gram=2)

Original text: 

анна каренина один из самых знаменитых романов льва толстого начинается ставшей афоризмом фразой все 

Encrypted text: 

яулжёшагттооюайцоодхюсппеьйеищчнттсзчъбжпжяущает июабмщпбмбйимцчооёояеитьвчькуьыбрддшзйядфжюпаёълхпэ 

Decrypted text: 

вая пошил ита чтитствдпижд эикичл егтдсет вавовсжба тоьстоко смситктрж иь пеылерътанжатвотляасад нти 

Correctly decrypted percentage: 0.19100160052917864


#### Результат заметно ухудшился, даже визуально видно, что биграммы совмещены совершенно неправильно

## Part 3

Метод заключается в следующем: 
1. генерируем текущий перестановкой символов изначального алфавита (или совмещением целевого алфавита и случайной перестановки имеющегося)
2. генерируем предложенный шифр перестановкой двух букв текущего шифра
3. декодируем предложение, используя текущий и предложенный шифры
4. считаем логарифмы правдоподобия, основанные на статистике биграмм, предварительно посчитанных на большом текстовом корпусе
5. если предложенный шифр лучше, чем текущий, принимаем его за новый текущий
6. повторяем шаги 2-5 до сходимости

In [22]:
def get_cipher(alphabet=alphabet, new_alphabet=alphabet):
    new_alphabet = random.sample(list(new_alphabet), len(new_alphabet))
    cipher_dict = {char:new_char for char, new_char in zip_longest(alphabet, new_alphabet, fillvalue='')}
    return dict_to_ordered_sorted(cipher_dict)

def encrypt(text, cipher):
    return ''.join([cipher[char] for char in text])

def decrypt(ciphered_text, cipher):
    cipher_decrypt = {v: k for k, v in cipher.items()}
    return ''.join([cipher_decrypt[char] for char in ciphered_text])

In [23]:
def text_to_bigram(text):
    return [text[i]+text[j] for i, j in zip(range(len(text)-1), range(1, len(text)))]

In [24]:
def text_to_n_gram(text, n_gram=1):
    return [text[i:i+n_gram] for i in range(len(text) - (n_gram - 1))]

In [25]:
def calculate_n_gram_frequency(text_n_gram):
    n_gram_frequency = Counter(text_n_gram)
    total_count = sum(n_gram_frequency.values())
    
    for key in n_gram_frequency:
        n_gram_frequency[key] /= total_count
        
    return n_gram_frequency

In [26]:
def n_gram_prob(n_gram, n_gram_frequency, text_n_gram):
    prob = n_gram_frequency.get(n_gram)

    if not prob:
        return 1 / len(text_n_gram)
    
    return prob

In [27]:
def log_likelihood(text, bigram_frequency, text_bigrams, n_gram=2):
    text_bigram = text_to_n_gram(text, n_gram)
    log_probs = [np.log(n_gram_prob(bigram, bigram_frequency, text_bigrams)) for bigram in text_bigram]
    return sum(log_probs)

In [28]:
def swap(x):
    i, j = random.sample(list(x.keys()), k=2)
    x[i], x[j] = x[j], x[i]

    return x

In [29]:
def accept_proposed(log_likelihood_proposed, log_likelihood_current):
    if log_likelihood_proposed > log_likelihood_current:
        return True
    return np.random.rand() < np.exp(log_likelihood_proposed - log_likelihood_current)

In [30]:
def mcmc_decrypt(encrypted_text, bigram_frequency, text_bigrams, 
                 iterations = 5000, alphabet=alphabet, new_alphabet=alphabet, n_gram=2):
    current_cipher = get_cipher(alphabet, new_alphabet)
    
    for i in tqdm(range(iterations)):
        proposed_cipher = swap(copy.deepcopy(current_cipher))

        decrypted_text = decrypt(encrypted_text, current_cipher)
        decrypted_text_proposed = decrypt(encrypted_text, proposed_cipher)

        log_likelihood_proposed = log_likelihood(decrypted_text_proposed, bigram_frequency, text_bigrams, n_gram)
        log_likelihood_current = log_likelihood(decrypted_text, bigram_frequency, text_bigrams, n_gram)

        if accept_proposed(log_likelihood_proposed, log_likelihood_current):
            current_cipher = proposed_cipher
            
    return decrypted_text

In [62]:
def encrypt_decrypt(text, bigram_frequency, text_bigram,
                    encrypted=False, iterations=5000,
                    alphabet=alphabet, new_alphabet=alphabet,
                    n_gram=2, epochs=3):
    original_text = text
    
    if not encrypted:
        cipher = get_cipher()
        text = encrypt(text, cipher)
        
    decrypted_texts = []
    
    for epoch in range(epochs):
        decrypted_text = mcmc_decrypt(text, bigram_frequency,
                                      text_bigram, iterations=iterations,
                                      alphabet=alphabet, new_alphabet=new_alphabet, n_gram=n_gram)
        if not encrypted:
            accuracy = decryption_accuracy(original_text, decrypted_text)
            decrypted_texts.append((decrypted_text, accuracy))
            
        else:
            decryption_likelihood = log_likelihood(decrypted_text, bigram_frequency, text_bigram, n_gram)
            decrypted_texts.append((decrypted_text, decryption_likelihood))
            
    return max(decrypted_texts, key=itemgetter(1))

In [32]:
text_rus_bigram = text_to_n_gram(text_rus, n_gram=2)

In [33]:
bigram_frequency = calculate_n_gram_frequency(text_rus_bigram)

In [39]:
decrypted_text = encrypt_decrypt(test_string, bigram_frequency, text_rus_bigram, iterations=20000, epochs=5)

100%|████████████████████████████████████████████████████████| 20000/20000 [01:08<00:00, 290.48it/s]


In [40]:
print(decrypted_text[0][:200])
print()
print(f'Correctly decrypted percentage: {decrypted_text[1]}')

свойство или приспособление организма потеншиально имеюфее акаптивную приспособительную шенность теория преакапташии позволяет описать механизм смены чундший органов в прошессе эволюшии и разрецить па

Correctly decrypted percentage: 0.914535666218035


In [65]:
decrypted_text = encrypt_decrypt(test_text[:10000], bigram_frequency, text_rus_bigram, iterations=7000, epochs=3)

100%|███████████████████████████████████████████████████████████| 7000/7000 [02:38<00:00, 44.08it/s]


In [66]:
print(decrypted_text[0][:200])
print()
print(f'Correctly decrypted percentage: {decrypted_text[1]}')

анна каренина один из самых знаменитых романов льва толстого начинается ставшей афоризмом фразой все счастливые семьи похожи друг на друга каждая несчастливая семья несчастлива посвоему это книга о ве

Correctly decrypted percentage: 1.0


#### Качество расшифровки даже на небольшом тестовом тексте заметно улучшилось. Однако не всегда качественная расшифровка получается с первого раза

## Part 4

In [41]:
encrypted_text = 'დჳჵჂႨშႼႨშჂხჂჲდႨსႹႭჾႣჵისႼჰႨჂჵჂႨႲႹႧჲჂႨსႹႭჾႣჵისႼჰႨჲდႩჳჲႨჇႨႠჲႹქႹႨჳႹႹჱჶდსჂႽႨႩႹჲႹႭႼჰႨჵდქႩႹႨႲႭႹႧჂჲႣჲიႨჳႩႹႭდდႨშჳდქႹႨშႼႨშჳდႨჳხდჵႣჵჂႨႲႭႣშჂჵისႹႨჂႨႲႹჵჇႧჂჲდႨჾႣႩჳჂჾႣჵისႼჰႨჱႣჵჵႨეႣႨႲႹჳჵდხსდდႨႧდჲშდႭჲႹდႨეႣხႣსჂდႨႩჇႭჳႣႨႾႹჲႽႨႩႹსდႧსႹႨႽႨსჂႧდქႹႨსდႨႹჱდჶႣნ'

In [42]:
right_decryption = 'eсли вы видите нормальный или почти нормальный текст у этого сообщения который легко прочитать скорее всего вы всё сделали правильно и получите максимальный балл за последнее четвертое задание курса хотя конечно я ничего не обещаю'

In [43]:
new_alphabet = set(encrypted_text)

In [109]:
decrypted_text = encrypt_decrypt(encrypted_text, bigram_frequency, text_rus_bigram,
                                 iterations = 100000, alphabet=alphabet, new_alphabet=new_alphabet,
                                 encrypted=True, epochs=5)

100%|█████████████████████████████████████████████████████| 100000/100000 [00:58<00:00, 1699.43it/s]


In [112]:
print(decrypted_text[0])
print()
f'Correctly decrypted percentage: {decryption_accuracy(right_decryption, decrypted_text[0])}'

если вы вимите нордальный или почти нордальный текст у этого сообжения который легко прочитать скорее всего вы все смелали правильно и получите даксидальный балл за послемнее четвертое замание курса хотя конечно я ничего не обежаш



'Correctly decrypted percentage: 0.9434782608695652'

## Part 6

Прямым применением данной модели является вскрытие простейших криптографических шифров, основанных на сдвиге букв или на их переставновке. 

Мне еще пришло в голову, что такая модель может применяться в генетике. Обучая перестановки генов, целевое распределение будет приближаться к распределению генов, которые, например, влияют на появление какого-то заболевания. Но насколько это адекватное решение, я не знаю.

Также такую модель можно использовать для unsupervised text translation